[참조]

네이버 기사 댓글 코드 : https://kkkapuq.tistory.com/71

In [ ]:
# !pip install --upgrade Beautifulsoup4 selenium 

# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')


In [ ]:
import pandas as pd 

df = pd.read_csv('/content/gdrive/MyDrive/웹과텍스트마이닝/기말 프로젝트/Data/네이버_뉴스_코인_20220509_20220609.csv')

In [ ]:
# df에 기사별 댓글 추가하기 위해서 빈 열 생성

df["Comments"] = ""

In [ ]:
# 네이버 기사 댓글 링크로 변경
# df['Link'][0][:39] + 'comment/' + df['Link'][0][39:]

In [ ]:
# 라이브러리를 로드합니다.
from bs4 import BeautifulSoup
import requests
import re
import sys
import pprint
 

In [ ]:
# 여러 리스트들을 하나로 묶어 주는 함수입니다. 
def flatten(l):
    flatList = []
    for elem in l:
        # if an element of a list is a list
        # iterate over this list and add elements to flatList
        if type(elem) == list:
            for e in elem:
                flatList.append(e)
        else:
            flatList.append(elem)
    return flatList

In [ ]:

def naver_news_comment_scrapper(all_url):

    df = pd.read_csv('/content/gdrive/MyDrive/웹과텍스트마이닝/기말 프로젝트/Data/네이버_뉴스_코인_20220509_20220609.csv')

    all_comments = [] 
    for i in range(len(all_url)):

            # 댓글을 달 빈 리스트를 생성합니다.
            List = []

            # 모든기사의 댓글을 모은 리스트를 생성합니다. 
            allCommetns = []
            
            ## 네이버 뉴스 url을 입력합니다.
            url = all_url[i][:39] + 'comment/' + all_url[i][39:] # 네이버 기사 링크에서 기사 댓글 링크로 변경 
            
            # url = https://n.news.naver.com/mnews/article/comment/001/0013236507?sid=104

            oid = url.split('/')[6] # oid=001
            aid = url.split('/')[7].split('?')[0] # aid = 0013236507
            
            page = 1
            header = {"User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36",
                "referer": url,}
            
            while True:
                c_url = "https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news" + oid + "%2C" + aid + "&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=" + str(
                    page) + "&refresh=false&sort=FAVORITE"
                
                # 파싱하는 단계입니다.
                r = requests.get(c_url, headers=header)
                cont = BeautifulSoup(r.content, "html.parser")
                total_comm = str(cont).split('comment":')[1].split(",")[0]
            
                match = re.findall('"contents":([^\*]*),"userIdNo"', str(cont))
                # 댓글을 리스트에 중첩합니다.
                List.append(match)
                # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.
                if int(total_comm) <= ((page) * 20):
                    break
                else:
                    page += 1


            # 리스트 결과입니다.
            all_comments.append(List)
    
   
    return all_comments


In [ ]:
# 한 기사의 댓글이 20개씩 묶어있어서 기사마다 한번 더 flatten 시켜줍니다.

new_a = pd.Series(a).apply(flatten)

len(new_a)

In [ ]:
df['Comments'] = new_a 

In [ ]:
file_path =  "/content/gdrive/My Drive/웹과텍스트마이닝/기말 프로젝트/Data/"

file_name = "네이버_뉴스_" + '코인_기사별_댓글' + ".csv"

file_full_path = file_path + file_name

df.to_csv(file_full_path ,  index = False, encoding="utf-8-sig") # utf-8-sig로 해야함!  

In [ ]:
import pandas as pd 


